In [26]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel
import requests
from pydantic import BaseModel, Field
import json
from dotenv import load_dotenv

In [25]:
# Load Anthropic API key
load_dotenv()

True

In [27]:
# Pydantic models for FHIR metadata

class SearchParameter(BaseModel):
    """FHIR search parameter definition"""
    name: str
    type: str | None  # Can sometimes be None if special type
    documentation: str | None = None

class ResourceMetadata(BaseModel):
    """Metadata for a single FHIR resource type"""
    type: str
    profile: str | None = None
    interactions: list[str]  # ['read', 'search-type', 'create', etc.]
    search_params: list[SearchParameter]

class FHIRMetadata(BaseModel):
    """Complete FHIR server metadata response"""
    searchable_types: list[str] = Field(
        description="List of resource types that support search-type interaction"
    )
    resource_metadata: dict[str, ResourceMetadata] = Field(
        description="Full metadata for each resource type, keyed by type name"
    )
    fhir_version: str | None = None
    server_url: str

In [28]:
def fetch_searchable_resources(base_url: str = "https://r4.smarthealthit.org") -> FHIRMetadata:
    """
    Fetch searchable resource types from FHIR server metadata.

    Based on reference code that queries /metadata endpoint and filters
    resources with 'search-type' interaction capability.

    Args:
        base_url: FHIR server base URL (default: SMART Health IT R4 server)

    Returns:
        FHIRMetadata: Pydantic model containing searchable types and full resource metadata

    Raises:
        requests.RequestException: If network request fails
        ValueError: If response is invalid or missing required fields
    """
    # Query metadata endpoint
    metadata_url = f"{base_url}/metadata"

    try:
        response = requests.get(metadata_url, timeout=10)
        response.raise_for_status()
        capability_statement = response.json()
    except requests.RequestException as e:
        raise requests.RequestException(f"Failed to fetch metadata from {metadata_url}: {e}")

    # Validate response structure
    if not capability_statement.get('rest') or len(capability_statement['rest']) == 0:
        raise ValueError("Invalid CapabilityStatement: missing 'rest' array")

    if not capability_statement['rest'][0].get('resource'):
        raise ValueError("Invalid CapabilityStatement: missing 'resource' array in rest[0]")

    # Extract searchable resources (matching reference code logic)
    searchable_types: list[str] = []
    resource_metadata: dict[str, ResourceMetadata] = {}

    for resource in capability_statement['rest'][0]['resource']:
        resource_type = resource.get('type')
        if not resource_type:
            continue

        # Check if resource supports search-type interaction (reference line 74-79)
        interactions = resource.get('interaction', [])
        interaction_codes = [interaction.get('code') for interaction in interactions]

        if 'search-type' in interaction_codes:
            searchable_types.append(resource_type)

            # Parse search parameters and sort alphabetically (reference line 81-90)
            search_params: list[SearchParameter] = []
            for param in resource.get('searchParam', []):
                search_params.append(SearchParameter(
                    name=param.get('name'),
                    type=param.get('type'),
                    documentation=param.get('documentation')
                ))

            # Sort by name (matching reference code)
            search_params.sort(key=lambda p: p.name)

            # Create ResourceMetadata object
            resource_metadata[resource_type] = ResourceMetadata(
                type=resource_type,
                profile=resource.get('profile'),
                interactions=interaction_codes,
                search_params=search_params
            )

    return FHIRMetadata(
        searchable_types=searchable_types,
        resource_metadata=resource_metadata,
        fhir_version=capability_statement.get('fhirVersion'),
        server_url=base_url
    )

In [21]:
# Test the function
metadata = fetch_searchable_resources()

print(f"FHIR Version: {metadata.fhir_version}")
print(f"Server URL: {metadata.server_url}")
print(f"\nFound {len(metadata.searchable_types)} searchable resource types:")
print(metadata.searchable_types[:10])  # Show first 10

# Examine a specific resource
if 'Patient' in metadata.resource_metadata:
    patient_meta = metadata.resource_metadata['Patient']
    print(f"\nPatient resource:")
    print(f"  Interactions: {patient_meta.interactions}")
    print(f"  Number of search parameters: {len(patient_meta.search_params)}")
    print(f"  First 5 search parameters: {[p.name for p in patient_meta.search_params[:5]]}")

FHIR Version: 4.0.0
Server URL: https://r4.smarthealthit.org

Found 146 searchable resource types:
['Account', 'ActivityDefinition', 'AdverseEvent', 'AllergyIntolerance', 'Appointment', 'AppointmentResponse', 'AuditEvent', 'Basic', 'Binary', 'BiologicallyDerivedProduct']

Patient resource:
  Interactions: ['read', 'vread', 'update', 'patch', 'delete', 'history-instance', 'history-type', 'create', 'search-type']
  Number of search parameters: 25
  First 5 search parameters: ['_id', '_language', 'active', 'address', 'address-city']


In [29]:
# Pydantic model for Select Types Agent output

class SelectedResourceTypes(BaseModel):
    """Result of resource type selection from user query"""
    selected_types: list[str] = Field(
        default=[],
        description="List of selected resource types, ordered by relevance (most relevant first)"
    )
    confidence: float = Field(
        default=0.0,
        ge=0.0,
        le=1.0,
        description="Confidence score (0.0-1.0) for the selection"
    )
    reasoning: str = Field(
        description="Explanation of why these types were selected"
    )
    error: str | None = Field(
        default=None,
        description="Error message if type not found or not searchable"
    )

In [ ]:
class SelectTypesAgent:
    """Agent for selecting FHIR resource types from natural language queries"""

    def __init__(self, metadata: FHIRMetadata):
        """
        Initialize the agent with FHIR server metadata.

        Args:
            metadata: FHIRMetadata object containing available searchable types
        """
        self.metadata = metadata
        self.model = AnthropicModel('claude-opus-4-5')

        self.agent = Agent(
            model=self.model,
            output_type=SelectedResourceTypes,
            system_prompt=self._build_system_prompt()
        )

    def select_types(self, query: str) -> SelectedResourceTypes:
        """
        Analyze query and return selected resource types.

        Args:
            query: Natural language query from user

        Returns:
            SelectedResourceTypes with selected types, confidence, and reasoning
        """
        result = self.agent.run_sync(query)
        return result.output

    def _build_system_prompt(self) -> str:
        """Build dynamic system prompt with available types from metadata"""
        types_list = ", ".join(sorted(self.metadata.searchable_types))

        return f"""You are a FHIR resource type selector. Analyze user queries and select the appropriate FHIR resource type(s).

Available searchable resource types ({len(self.metadata.searchable_types)} total):
{types_list}

Your task:
1. Analyze the user's query to understand what data they want
2. Select the most appropriate resource type(s) from the available list above
3. Return types in order of relevance (most relevant first)
4. Provide a confidence score (0.0-1.0) and reasoning

Confidence scoring guidelines:
- 0.9-1.0: Exact type name mentioned or very clear semantic match
- 0.7-0.9: Clear semantic match with good context
- 0.5-0.7: Reasonable match but some ambiguity
- 0.3-0.5: Multiple valid options, unclear which is best
- 0.0-0.3: Very uncertain or no good matches

Common mappings:
- "patients", "patient demographics", "people" → Patient
- "vital signs", "blood pressure", "lab results", "observations" → Observation
- "medications", "prescriptions", "drugs" → Medication, MedicationRequest
- "encounters", "visits", "appointments" → Encounter
- "procedures", "surgeries", "operations" → Procedure
- "conditions", "diagnoses", "problems", "diseases" → Condition
- "allergies" → AllergyIntolerance
- "immunizations", "vaccinations" → Immunization

Error handling:
- If requested type doesn't exist in available list: return empty selected_types, set error message with suggestion
- If type exists but query is ambiguous: return multiple types with lower confidence
- If query is too vague: return most likely types with low confidence and explain in reasoning

IMPORTANT: Only select types from the available list above. Do not make up or suggest types that aren't in the list.

Always explain your reasoning clearly."""

In [ ]:
# Initialize the SelectTypesAgent
select_agent = SelectTypesAgent(metadata)

print("SelectTypesAgent initialized successfully!")
print(f"Agent has access to {len(metadata.searchable_types)} searchable resource types")

In [ ]:
# Test 1: Clear, specific query
print("=" * 60)
print("TEST 1: Clear query - 'Find all patients born after 1990'")
print("=" * 60)
result = select_agent.select_types("Find all patients born after 1990")
print(f"Selected types: {result.selected_types}")
print(f"Confidence: {result.confidence:.2f}")
print(f"Reasoning: {result.reasoning}")
print(f"Error: {result.error}")
print()

In [ ]:
# Test 2: Ambiguous query (multiple matching types)
print("=" * 60)
print("TEST 2: Ambiguous query - 'Get medication data'")
print("=" * 60)
result = select_agent.select_types("Get medication data")
print(f"Selected types: {result.selected_types}")
print(f"Confidence: {result.confidence:.2f}")
print(f"Reasoning: {result.reasoning}")
print(f"Error: {result.error}")
print()

In [ ]:
# Test 3: Semantic query (blood pressure -> Observation)
print("=" * 60)
print("TEST 3: Semantic query - 'Show me blood pressure readings'")
print("=" * 60)
result = select_agent.select_types("Show me blood pressure readings")
print(f"Selected types: {result.selected_types}")
print(f"Confidence: {result.confidence:.2f}")
print(f"Reasoning: {result.reasoning}")
print(f"Error: {result.error}")
print()

In [ ]:
# Test 4: Non-existent type
print("=" * 60)
print("TEST 4: Non-existent type - 'Find XYZ records'")
print("=" * 60)
result = select_agent.select_types("Find XYZ records")
print(f"Selected types: {result.selected_types}")
print(f"Confidence: {result.confidence:.2f}")
print(f"Reasoning: {result.reasoning}")
print(f"Error: {result.error}")
print()

In [4]:
model = AnthropicModel('claude-opus-4-5')


UserError: Set the `ANTHROPIC_API_KEY` environment variable or pass it via `AnthropicProvider(api_key=...)`to use the Anthropic provider.